In [33]:
# ============================================
# 📌 PART A — TRAINING (train.csv만 사용)
# ============================================

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import xgboost as xgb
from tqdm import tqdm

# -------------------------------------------------------
# 📌 0. Seed 고정
# -------------------------------------------------------
import random
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LOOKBACK = 28
EPOCHS = 40
BATCH = 32

# -------------------------------------------------------
# 0. 문자열 숫자 → float 변환 (콤마 제거)
# -------------------------------------------------------
def clean_numeric(df):
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].astype(str).str.replace(",", "")
            df[col] = pd.to_numeric(df[col], errors="ignore")
    return df

# --------------------------------------------
# 1. train 데이터만 로드
# --------------------------------------------
TRAIN_PATH = "/content/drive/MyDrive/기계학습/팀플/Data/POS_train_val.csv"
ACAD_PATH  = "/content/drive/MyDrive/기계학습/팀플/Data/학사일정_정리(2325).csv"

train = pd.read_csv(TRAIN_PATH)
acad  = pd.read_csv(ACAD_PATH)

train["영업일자"] = pd.to_datetime(train["영업일자"])
acad["영업일자"] = pd.to_datetime(acad["date"])
acad = acad.drop(columns=["date"])

train = train.merge(acad, on="영업일자", how="left")
train = train.sort_values("영업일자").reset_index(drop=True)

# 일매출 정수 변환
train["일매출"] = (
    train["일매출"].astype(str)
        .str.replace(",", "")
        .str.replace(" ", "")
)
train = clean_numeric(train)   # 🔥 object → float 변환

# --------------------------------------------
# 2. Feature Engineering
# --------------------------------------------
def make_features(df):
    df = df.copy()
    df["DayOfWeek"] = df["영업일자"].dt.dayofweek
    df["Month"] = df["영업일자"].dt.month
    df["Day"] = df["영업일자"].dt.day
    df["IsWeekend"] = (df["DayOfWeek"] >= 5).astype(int)
    df["OpHoursFactor"] = df["DayOfWeek"].apply(lambda x: 1 if x<5 else (0.55 if x==5 else 0))

    for lag in [1,2,3,7,14,28]:
        df[f"Lag{lag}"] = df["일매출"].shift(lag)

    for w in [7,14,28]:
        df[f"Mean{w}"] = df["일매출"].rolling(w).mean()
        df[f"Std{w}"]  = df["일매출"].rolling(w).std()

    return df.fillna(0)

train = make_features(train)

# --------------------------------------------
# 3. Train/Validation Split
# --------------------------------------------
train_df, val_df = train_test_split(train, test_size=0.2, shuffle=False)

# --------------------------------------------
# 4. meta_cols = 숫자형 only (object 제거!)
# --------------------------------------------
numeric_cols = train.select_dtypes(include=[np.number]).columns.tolist()
meta_cols = [c for c in numeric_cols if c not in ["일매출"]]

# --------------------------------------------
# 5. Scaling (train에만 fit)
# --------------------------------------------
sales_scaler = MinMaxScaler()
meta_scaler  = MinMaxScaler()

train_df["sales_scaled"] = sales_scaler.fit_transform(train_df[["일매출"]])
train_df[meta_cols] = meta_scaler.fit_transform(train_df[meta_cols])

val_df["sales_scaled"] = sales_scaler.transform(val_df[["일매출"]])
val_df[meta_cols] = meta_scaler.transform(val_df[meta_cols])

# --------------------------------------------
# 6. Sequence Dataset
# --------------------------------------------
def create_seq(df):
    X, y = [], []
    sales = df["sales_scaled"].values
    meta  = df[meta_cols].values

    for i in range(len(df)-LOOKBACK):
        seq = np.concatenate([
            sales[i:i+LOOKBACK].reshape(LOOKBACK,1),
            meta[i:i+LOOKBACK]
        ], axis=1)
        X.append(seq)
        y.append(sales[i+LOOKBACK])
    return np.array(X), np.array(y)

X_train, y_train = create_seq(train_df)
X_val,   y_val   = create_seq(val_df)

X_train = torch.tensor(X_train, dtype=torch.float32).to(DEVICE)
y_train = torch.tensor(y_train, dtype=torch.float32).to(DEVICE)
X_val   = torch.tensor(X_val,   dtype=torch.float32).to(DEVICE)
y_val   = torch.tensor(y_val,   dtype=torch.float32).to(DEVICE)

# --------------------------------------------
# 7. Models
# --------------------------------------------
input_dim = 1 + len(meta_cols)

class MetaLSTM(nn.Module):
    def __init__(self, input_dim, hidden=64):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden, num_layers=2, batch_first=True)
        self.fc = nn.Linear(hidden, 1)
    def forward(self, x): return self.fc(self.lstm(x)[0][:,-1])

class MetaGRU(nn.Module):
    def __init__(self, input_dim, hidden=64):
        super().__init__()
        self.gru = nn.GRU(input_dim, hidden, num_layers=2, batch_first=True)
        self.fc = nn.Linear(hidden, 1)
    def forward(self, x): return self.fc(self.gru(x)[0][:,-1])

lstm = MetaLSTM(input_dim).to(DEVICE)
gru  = MetaGRU(input_dim).to(DEVICE)

opt_l = torch.optim.Adam(lstm.parameters(), lr=0.001)
opt_g = torch.optim.Adam(gru.parameters(),  lr=0.001)
criterion = nn.MSELoss()

# --------------------------------------------
# 8. Train NN Models
# --------------------------------------------
for ep in range(EPOCHS):
    pred_l = lstm(X_train)
    pred_g = gru(X_train)

    loss_l = criterion(pred_l.squeeze(), y_train)
    loss_g = criterion(pred_g.squeeze(), y_train)

    opt_l.zero_grad(); loss_l.backward(); opt_l.step()
    opt_g.zero_grad(); loss_g.backward(); opt_g.step()

    print(f"[Epoch {ep+1}] LSTM={loss_l.item():.4f}, GRU={loss_g.item():.4f}")

# --------------------------------------------
# 9. Validation Evaluation
# --------------------------------------------
def evaluate(name, y_true, y_pred):

    # --- y_true 처리 ---
    if isinstance(y_true, torch.Tensor):
        y_true = y_true.detach().cpu().numpy()
    else:
        y_true = np.array(y_true)

    # --- y_pred 처리 ---
    if isinstance(y_pred, torch.Tensor):
        y_pred = y_pred.detach().cpu().numpy()
    else:
        y_pred = np.array(y_pred)

    # --- 지표 계산 ---
    mae = np.mean(np.abs(y_true - y_pred))
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    smape = np.mean(
        np.abs(y_true - y_pred) /
        ((np.abs(y_true)+np.abs(y_pred))/2 + 1e-9)
    ) * 100

    print(f"\n=== {name} Validation ===")
    print(f"MAE   : {mae:.4f}")
    print(f"RMSE  : {rmse:.4f}")
    print(f"SMAPE : {smape:.2f}%")



lstm.eval(); gru.eval()
evaluate("LSTM", y_val, lstm(X_val))
evaluate("GRU",  y_val, gru(X_val))

# --------------------------------------------
# 10. Tree Models (train 내부에서만)
# --------------------------------------------
tree_features = meta_cols  # safe

lgb_train = lgb.Dataset(train_df[tree_features], label=train_df["일매출"])
lgb_model = lgb.train({"objective":"regression","learning_rate":0.03}, lgb_train, 300)

dtrain = xgb.DMatrix(train_df[tree_features], label=train_df["일매출"])
xgb_model = xgb.train({"objective":"reg:squarederror","eta":0.03}, dtrain, 300)

# --------------------------------------------
# Validation (길이 맞춰서 평가)
# --------------------------------------------
pred_lgb_val = lgb_model.predict(val_df[tree_features])
pred_xgb_val = xgb_model.predict(xgb.DMatrix(val_df[tree_features]))

valid_len = len(y_val)

# LightGBM
pred_lgb_trim = pred_lgb_val[-valid_len:]
pred_lgb_scaled = sales_scaler.transform(pred_lgb_trim.reshape(-1,1)).reshape(-1)
evaluate("LightGBM", y_val, pred_lgb_scaled)

# XGBoost
pred_xgb_trim = pred_xgb_val[-valid_len:]
pred_xgb_scaled = sales_scaler.transform(pred_xgb_trim.reshape(-1,1)).reshape(-1)
evaluate("XGBoost", y_val, pred_xgb_scaled)


print("\n\n============================")
print("PART A 종료 (train만 사용, 데이터 누수 ZERO)")
print("============================\n")


/tmp/ipython-input-3850780896.py:41: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")
/tmp/ipython-input-3850780896.py:41: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")
/tmp/ipython-input-3850780896.py:41: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")


[Epoch 1] LSTM=0.0584, GRU=0.1223
[Epoch 2] LSTM=0.0485, GRU=0.0728
[Epoch 3] LSTM=0.0417, GRU=0.0434
[Epoch 4] LSTM=0.0378, GRU=0.0342
[Epoch 5] LSTM=0.0364, GRU=0.0414
[Epoch 6] LSTM=0.0369, GRU=0.0501
[Epoch 7] LSTM=0.0378, GRU=0.0508
[Epoch 8] LSTM=0.0379, GRU=0.0455
[Epoch 9] LSTM=0.0369, GRU=0.0390
[Epoch 10] LSTM=0.0356, GRU=0.0342
[Epoch 11] LSTM=0.0343, GRU=0.0319
[Epoch 12] LSTM=0.0334, GRU=0.0319
[Epoch 13] LSTM=0.0328, GRU=0.0331
[Epoch 14] LSTM=0.0325, GRU=0.0345
[Epoch 15] LSTM=0.0324, GRU=0.0354
[Epoch 16] LSTM=0.0322, GRU=0.0355
[Epoch 17] LSTM=0.0319, GRU=0.0348
[Epoch 18] LSTM=0.0315, GRU=0.0335
[Epoch 19] LSTM=0.0309, GRU=0.0321
[Epoch 20] LSTM=0.0302, GRU=0.0306
[Epoch 21] LSTM=0.0295, GRU=0.0295
[Epoch 22] LSTM=0.0290, GRU=0.0289
[Epoch 23] LSTM=0.0287, GRU=0.0288
[Epoch 24] LSTM=0.0286, GRU=0.0290
[Epoch 25] LSTM=0.0285, GRU=0.0293
[Epoch 26] LSTM=0.0283, GRU=0.0294
[Epoch 27] LSTM=0.0278, GRU=0.0293
[Epoch 28] LSTM=0.0273, GRU=0.0288
[Epoch 29] LSTM=0.0269, GRU=0

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [34]:
# ============================================
# 📌 PART B — TEST 예측 + 평가지표 계산 (완전수정본)
# ============================================

# -------------------------------
# 1. TEST 로드
# -------------------------------
TEST_PATH = "/content/drive/MyDrive/기계학습/팀플/Data/POS_test.csv"

test = pd.read_csv(TEST_PATH)
test["영업일자"] = pd.to_datetime(test["영업일자"])

# 숫자문자 제거
test = clean_numeric(test)

# 학사일정 merge
test = test.merge(acad, on="영업일자", how="left")

# train에서 사용한 meta_cols가 test에 없으면 0 생성
for col in meta_cols:
    if col not in test.columns:
        test[col] = 0

# -------------------------------
# 2. Feature Engineering (train과 동일)
# -------------------------------
test = make_features(test)

# -------------------------------
# 3. Scaling
# -------------------------------
test_scaled = test.copy()
test_scaled["sales_scaled"] = sales_scaler.transform(test_scaled[["일매출"]])
test_scaled[meta_cols] = meta_scaler.transform(test_scaled[meta_cols])


# ======================================================
# 4. NN 전용 — full-length rolling prediction 함수
# ======================================================
def predict_nn_full(model, train_df, test_df):
    model.eval()
    history = pd.concat([train_df, test_df], ignore_index=True)
    preds = []

    for i in range(len(test_df)):
        start = len(train_df) + i - LOOKBACK
        end   = start + LOOKBACK

        seq_sales = history["sales_scaled"].values[start:end].reshape(LOOKBACK,1)
        seq_meta  = history[meta_cols].values[start:end]

        seq = np.concatenate([seq_sales, seq_meta], axis=1)

        X = torch.tensor(seq, dtype=torch.float32).unsqueeze(0).to(DEVICE)

        with torch.no_grad():
            scaled_pred = model(X).item()

        # 역스케일링 → raw 매출
        real_pred = sales_scaler.inverse_transform([[scaled_pred]])[0,0]
        preds.append(real_pred)

    return np.array(preds)


# -------------------------------
# 5. LSTM/GRU 예측
# -------------------------------
pred_lstm = predict_nn_full(lstm, train_df, test_scaled)
pred_gru  = predict_nn_full(gru,  train_df, test_scaled)


# -------------------------------
# 6. Tree 모델 예측 (raw)
# -------------------------------
pred_lgb = lgb_model.predict(test_scaled[tree_features])
pred_xgb = xgb_model.predict(xgb.DMatrix(test_scaled[tree_features]))

# raw 그대로 pTree 계산
pTree_raw = (pred_lgb + pred_xgb) / 2


# ======================================================
# 7. 길이 동기화 (ROLLING 방식 차이 방지)
# ======================================================
min_len = min(len(pred_lstm), len(pred_gru), len(pTree_raw))

pred_lstm = pred_lstm[-min_len:]
pred_gru  = pred_gru[-min_len:]
pred_lgb  = pred_lgb[-min_len:]
pred_xgb  = pred_xgb[-min_len:]
pTree_raw = pTree_raw[-min_len:]

# y_true 생성
y_true = test["일매출"].values[-min_len:]


# -------------------------------
# 8. 최종 앙상블 (raw 단위)
# -------------------------------
final_pred = 0.7 * pTree_raw + 0.3 * ((pred_lstm + pred_gru) / 2)


# -------------------------------
# 9. 평가지표 계산
# -------------------------------
def MAE(a, b):   return np.mean(np.abs(a-b))
def RMSE(a, b):  return np.sqrt(np.mean((a-b)**2))
def SMAPE(a, b): return np.mean(2 * np.abs(a-b) / (np.abs(a)+np.abs(b)+1e-9)) * 100

print("\n==================== TEST 성능 ====================")
print(f"LSTM MAE   : {MAE(y_true, pred_lstm):.2f}")
print(f"GRU  MAE   : {MAE(y_true, pred_gru):.2f}")
print(f"LGB  MAE   : {MAE(y_true, pred_lgb):.2f}")
print(f"XGB  MAE   : {MAE(y_true, pred_xgb):.2f}")
print(f"pTree MAE  : {MAE(y_true, pTree_raw):.2f}")

print("\n--- Final Ensemble ---")
print(f"MAE   : {MAE(y_true, final_pred):.2f}")
print(f"RMSE  : {RMSE(y_true, final_pred):.2f}")
print(f"SMAPE : {SMAPE(y_true, final_pred):.2f}%")
print("=====================================================\n")


# -------------------------------
# 10. CSV 저장
# -------------------------------
output = pd.DataFrame({
    "영업일자": test["영업일자"].values[-min_len:],
    "실제매출": y_true,
    "예측매출": final_pred
})

output.to_csv("final_test_prediction.csv", index=False)
print("📌 저장 완료 → final_test_prediction.csv")
print(output.head())


/tmp/ipython-input-3850780896.py:41: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")
/tmp/ipython-input-3850780896.py:41: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")



==================== TEST 성능 ====================
LSTM MAE   : 98615.46
GRU  MAE   : 95370.05
LGB  MAE   : 3370.71
XGB  MAE   : 3506.88
pTree MAE  : 2994.01

--- Final Ensemble ---
MAE   : 28781.98
RMSE  : 37577.16
SMAPE : 77.78%

📌 저장 완료 → final_test_prediction.csv
        영업일자    실제매출           예측매출
0 2025-08-10       0   76189.160672
1 2025-08-11       0   61231.909474
2 2025-08-12   81300  111065.960756
3 2025-08-13  114900  118451.178547
4 2025-08-14       0   29926.535807


**pTree만 앙상블**

In [35]:
# ============================================
# 📌 PART B — TEST 예측 + 평가지표 계산 (pTree-only ensemble)
# ============================================

# -------------------------------
# 1. TEST 로드
# -------------------------------
TEST_PATH = "/content/drive/MyDrive/기계학습/팀플/Data/POS_test.csv"

test = pd.read_csv(TEST_PATH)
test["영업일자"] = pd.to_datetime(test["영업일자"])

# 숫자문자 제거
test = clean_numeric(test)

# 학사일정 merge
test = test.merge(acad, on="영업일자", how="left")

# train에서 사용한 meta_cols가 test에 없으면 0 생성
for col in meta_cols:
    if col not in test.columns:
        test[col] = 0

# -------------------------------
# 2. Feature Engineering (train과 동일)
# -------------------------------
test = make_features(test)

# -------------------------------
# 3. Scaling
# -------------------------------
test_scaled = test.copy()
test_scaled["sales_scaled"] = sales_scaler.transform(test_scaled[["일매출"]])
test_scaled[meta_cols] = meta_scaler.transform(test_scaled[meta_cols])


# ======================================================
# 4. NN 전용 — full-length rolling prediction 함수
# ======================================================
def predict_nn_full(model, train_df, test_df):
    model.eval()
    history = pd.concat([train_df, test_df], ignore_index=True)
    preds = []

    for i in range(len(test_df)):
        start = len(train_df) + i - LOOKBACK
        end   = start + LOOKBACK

        seq_sales = history["sales_scaled"].values[start:end].reshape(LOOKBACK,1)
        seq_meta  = history[meta_cols].values[start:end]

        seq = np.concatenate([seq_sales, seq_meta], axis=1)

        X = torch.tensor(seq, dtype=torch.float32).unsqueeze(0).to(DEVICE)

        with torch.no_grad():
            scaled_pred = model(X).item()

        # scaled → raw 매출
        real_pred = sales_scaler.inverse_transform([[scaled_pred]])[0,0]
        preds.append(real_pred)

    return np.array(preds)


# -------------------------------
# 5. LSTM/GRU 예측
# -------------------------------
pred_lstm = predict_nn_full(lstm, train_df, test_scaled)
pred_gru  = predict_nn_full(gru,  train_df, test_scaled)


# -------------------------------
# 6. Tree 모델 예측 (raw)
# -------------------------------
pred_lgb = lgb_model.predict(test_scaled[tree_features])
pred_xgb = xgb_model.predict(xgb.DMatrix(test_scaled[tree_features]))

# raw 그대로 pTree 계산
pTree_raw = (pred_lgb + pred_xgb) / 2


# ======================================================
# 7. 길이 동기화 (ROLLING 방식 차이 방지)
# ======================================================
min_len = min(len(pred_lstm), len(pred_gru), len(pTree_raw))

pred_lstm = pred_lstm[-min_len:]
pred_gru  = pred_gru[-min_len:]
pred_lgb  = pred_lgb[-min_len:]
pred_xgb  = pred_xgb[-min_len:]
pTree_raw = pTree_raw[-min_len:]

# y_true 생성 (LOOKBACK 이후 구간)
y_true = test["일매출"].values[-min_len:]


# -------------------------------
# 8. 최종 앙상블 (🔥 pTree만 사용)
# -------------------------------
# 기존 코드:
# final_pred = 0.7 * pTree_raw + 0.3 * ((pred_lstm + pred_gru) / 2)

# 👉 수정: pTree 단독 사용
final_pred = pTree_raw.copy()


# -------------------------------
# 9. 평가지표 계산
# -------------------------------
def MAE(a, b):   return np.mean(np.abs(a-b))
def RMSE(a, b):  return np.sqrt(np.mean((a-b)**2))
def SMAPE(a, b): return np.mean(2 * np.abs(a-b) / (np.abs(a)+np.abs(b)+1e-9)) * 100

print("\n==================== TEST 성능 ====================")
print(f"LSTM MAE   : {MAE(y_true, pred_lstm):.2f}")
print(f"GRU  MAE   : {MAE(y_true, pred_gru):.2f}")
print(f"LGB  MAE   : {MAE(y_true, pred_lgb):.2f}")
print(f"XGB  MAE   : {MAE(y_true, pred_xgb):.2f}")
print(f"pTree MAE  : {MAE(y_true, pTree_raw):.2f}")

print("\n--- Final Ensemble (pTree only) ---")
print(f"MAE   : {MAE(y_true, final_pred):.2f}")
print(f"RMSE  : {RMSE(y_true, final_pred):.2f}")
print(f"SMAPE : {SMAPE(y_true, final_pred):.2f}%")
print("=====================================================\n")


# -------------------------------
# 10. CSV 저장
# -------------------------------
output = pd.DataFrame({
    "영업일자": test["영업일자"].values[-min_len:],
    "실제매출": y_true,
    "예측매출": final_pred
})

output.to_csv("final_test_prediction.csv", index=False)
print("📌 저장 완료 → final_test_prediction.csv")
print(output.head())


/tmp/ipython-input-3850780896.py:41: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")
/tmp/ipython-input-3850780896.py:41: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")



==================== TEST 성능 ====================
LSTM MAE   : 98615.46
GRU  MAE   : 95370.05
LGB  MAE   : 3370.71
XGB  MAE   : 3506.88
pTree MAE  : 2994.01

--- Final Ensemble (pTree only) ---
MAE   : 2994.01
RMSE  : 5279.21
SMAPE : 73.52%

📌 저장 완료 → final_test_prediction.csv
        영업일자    실제매출           예측매출
0 2025-08-10       0     147.632603
1 2025-08-11       0     -12.215185
2 2025-08-12   81300   86448.554873
3 2025-08-13  114900  112419.851372
4 2025-08-14       0      52.299545
